[Link a la carpeta con todo el TP](https://drive.google.com/drive/u/0/folders/1do-iyf2SzQln-fh8tmu9mSfzLuim5xs5)

# Imports & Download stage

In [7]:
!pip install category_encoders

In [8]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
from category_encoders import BinaryEncoder
import pandas as pd
import numpy as np
%matplotlib inline
import pyarrow.parquet as pq
import os
import random

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDClassifier
from scipy.stats import uniform, randint
from sklearn.metrics import f1_score

pd.options.display.float_format = '{:20,.10f}'.format

import gc

In [9]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [10]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

ACLARACIÓN: Como usé el mismo feature engineering para ambos modelos, lo hice en [otro colab](https://colab.research.google.com/drive/1oolZwdMmiiPSBROS8q7c9taycd5NoJgR#scrollTo=XeHTXwYzXc3o) y me guardé ese dataset (que es el que estoy cargando aquí)

In [11]:
if not os.path.exists('test.parq'):
  id1="18FlKfUK0ItadAP1jz9KG96KrbWyiQ1zo"
  downloaded1 = drive.CreateFile({'id': id1})
  downloaded1.GetContentFile('test.parq')

if not os.path.exists('train.parq'):
  id2="1H67jcnBBH5eA6elQu95fEDQwWzOuK5Gr"
  downloaded2 = drive.CreateFile({'id': id2})
  downloaded2.GetContentFile('train.parq')

# Init stage

load dataframe

In [12]:
raw_df = pq.read_table('train.parq').to_pandas()
val_df = pq.read_table('test.parq').to_pandas()

Train/test split based on IPs

In [13]:
# First, let's create a list of unique IP addresses
unique_ips = raw_df['attacker_ip_enum'].unique()

# Split the unique IPs into two sets with roughly 80%-20% proportions
train_ips, test_ips = train_test_split(unique_ips, test_size=0.2, random_state=317)

# Use these sets to filter the original DataFrame
train_df = raw_df[raw_df['attacker_ip_enum'].isin(train_ips)]
test_df = raw_df[raw_df['attacker_ip_enum'].isin(test_ips)]

del raw_df
del unique_ips
gc.collect()

8

# Encoding stage

Para el encoding traté de mantener la mayor cantidad de información posible de los dataframes. Por eso sólo usé mean cuando había demasiadas categorías.

Además, las categorías mean-encodeadas en el preceptron eran muy importantes, por lo que me pareció que beneficiaría al modelo tenerlas así

Como la única categoría que no encodeé con mean/OHE/binary fue el día que ya venía encodeado del [otro colab](https://colab.research.google.com/drive/1oolZwdMmiiPSBROS8q7c9taycd5NoJgR#scrollTo=x6iqMOh3yWUk) con seno y coseno, no hizo falta normalizar las features

### Drop IP to avoid overfitting/target leak

In [14]:
train_df.drop(['attacker_ip_enum'], axis=1, inplace=True)
test_df.drop(['attacker_ip_enum'], axis=1, inplace=True)

val_ips = val_df['attacker_ip_enum'] # I'll need these for kaggle submission
val_df.drop(['attacker_ip_enum'], axis=1, inplace=True)

### Countries (binary)

In [15]:
# Initialize a BinaryEncoder for the 'watcher_country' column
binary_encoder = BinaryEncoder(cols=['watcher_country'])

# Fit and transform on the training data
train_df_encoded = binary_encoder.fit_transform(train_df[['watcher_country']])

# Transform the test data
test_df_encoded = binary_encoder.transform(test_df[['watcher_country']])
val_df_encoded = binary_encoder.transform(val_df[['watcher_country']])

# Merge the encoded columns back into the original DataFrames
train_df = pd.concat([train_df, train_df_encoded], axis=1)
test_df = pd.concat([test_df, test_df_encoded], axis=1)
val_df = pd.concat([val_df, val_df_encoded], axis=1)

# Clean up unnecessary variables
del train_df_encoded, test_df_encoded, val_df_encoded
gc.collect()

# Drop the original 'watcher_country' column
train_df.drop('watcher_country', axis=1, inplace=True)
test_df.drop('watcher_country', axis=1, inplace=True)
val_df.drop('watcher_country', axis=1, inplace=True)

# Cast to uint8 for efficiency
train_df['watcher_country_0'] = train_df['watcher_country_0'].astype('uint8')
test_df['watcher_country_0'] = test_df['watcher_country_0'].astype('uint8')
val_df['watcher_country_0'] = val_df['watcher_country_0'].astype('uint8')

train_df['watcher_country_1'] = train_df['watcher_country_1'].astype('uint8')
test_df['watcher_country_1'] = test_df['watcher_country_1'].astype('uint8')
val_df['watcher_country_1'] = val_df['watcher_country_1'].astype('uint8')

train_df['watcher_country_2'] = train_df['watcher_country_2'].astype('uint8')
test_df['watcher_country_2'] = test_df['watcher_country_2'].astype('uint8')
val_df['watcher_country_2'] = val_df['watcher_country_2'].astype('uint8')

train_df['watcher_country_3'] = train_df['watcher_country_3'].astype('uint8')
test_df['watcher_country_3'] = test_df['watcher_country_3'].astype('uint8')
val_df['watcher_country_3'] = val_df['watcher_country_3'].astype('uint8')

train_df['watcher_country_4'] = train_df['watcher_country_4'].astype('uint8')
test_df['watcher_country_4'] = test_df['watcher_country_4'].astype('uint8')
val_df['watcher_country_4'] = val_df['watcher_country_4'].astype('uint8')

In [16]:
# Initialize a BinaryEncoder for the 'attacker_country' column
binary_encoder = BinaryEncoder(cols=['attacker_country'])

# Fit and transform on the training data
train_df_encoded = binary_encoder.fit_transform(train_df[['attacker_country']])

# Transform the test data
test_df_encoded = binary_encoder.transform(test_df[['attacker_country']])
val_df_encoded = binary_encoder.transform(val_df[['attacker_country']])

# Merge the encoded columns back into the original DataFrames
train_df = pd.concat([train_df, train_df_encoded], axis=1)
test_df = pd.concat([test_df, test_df_encoded], axis=1)
val_df = pd.concat([val_df, val_df_encoded], axis=1)

# Clean up unnecessary variables
del train_df_encoded, test_df_encoded, val_df_encoded
gc.collect()

# Drop the original 'attacker_country' column
train_df.drop('attacker_country', axis=1, inplace=True)
test_df.drop('attacker_country', axis=1, inplace=True)
val_df.drop('attacker_country', axis=1, inplace=True)

# Cast to uint8 for efficiency
train_df['attacker_country_0'] = train_df['attacker_country_0'].astype('uint8')
test_df['attacker_country_0'] = test_df['attacker_country_0'].astype('uint8')
val_df['attacker_country_0'] = val_df['attacker_country_0'].astype('uint8')

train_df['attacker_country_1'] = train_df['attacker_country_1'].astype('uint8')
test_df['attacker_country_1'] = test_df['attacker_country_1'].astype('uint8')
val_df['attacker_country_1'] = val_df['attacker_country_1'].astype('uint8')

train_df['attacker_country_2'] = train_df['attacker_country_2'].astype('uint8')
test_df['attacker_country_2'] = test_df['attacker_country_2'].astype('uint8')
val_df['attacker_country_2'] = val_df['attacker_country_2'].astype('uint8')

train_df['attacker_country_3'] = train_df['attacker_country_3'].astype('uint8')
test_df['attacker_country_3'] = test_df['attacker_country_3'].astype('uint8')
val_df['attacker_country_3'] = val_df['attacker_country_3'].astype('uint8')

train_df['attacker_country_4'] = train_df['attacker_country_4'].astype('uint8')
test_df['attacker_country_4'] = test_df['attacker_country_4'].astype('uint8')
val_df['attacker_country_4'] = val_df['attacker_country_4'].astype('uint8')

### AS Names (Mean/Target)

In [17]:
target_encoder = TargetEncoder(cols=['watcher_as_name'])

train_df_encoded = target_encoder.fit_transform(train_df[['watcher_as_name']], train_df['label'])
test_df_encoded = target_encoder.transform(test_df[['watcher_as_name']])
val_df_encoded = target_encoder.transform(val_df[['watcher_as_name']])

train_df[['watcher_as_name']] = train_df_encoded.astype('float32')
test_df[['watcher_as_name']] = test_df_encoded.astype('float32')
val_df[['watcher_as_name']] = val_df_encoded.astype('float32')

del train_df_encoded, test_df_encoded, val_df_encoded, target_encoder
gc.collect()

0

In [18]:
target_encoder = TargetEncoder(cols=['attacker_as_name'])

train_df_encoded = target_encoder.fit_transform(train_df[['attacker_as_name']], train_df['label'])
test_df_encoded = target_encoder.transform(test_df[['attacker_as_name']])
val_df_encoded = target_encoder.transform(val_df[['attacker_as_name']])

train_df[['attacker_as_name']] = train_df_encoded.astype('float32')
test_df[['attacker_as_name']] = test_df_encoded.astype('float32')
val_df[['attacker_as_name']] = val_df_encoded.astype('float32')

del train_df_encoded, test_df_encoded, val_df_encoded, target_encoder
gc.collect()

0

### Watcher UUID Enums (Mean/Target)

In [19]:
target_encoder = TargetEncoder(cols=['watcher_uuid_enum'])

train_df_encoded = target_encoder.fit_transform(train_df[['watcher_uuid_enum']], train_df['label'])
test_df_encoded = target_encoder.transform(test_df[['watcher_uuid_enum']])
val_df_encoded = target_encoder.transform(val_df[['watcher_uuid_enum']])

train_df[['watcher_uuid_enum']] = train_df_encoded.astype('float32')
test_df[['watcher_uuid_enum']] = test_df_encoded.astype('float32')
val_df[['watcher_uuid_enum']] = val_df_encoded.astype('float32')

del train_df_encoded, test_df_encoded, val_df_encoded, target_encoder
gc.collect()

0

In [20]:
target_encoder = TargetEncoder(cols=['attacker_as_name'])

train_df_encoded = target_encoder.fit_transform(train_df[['attacker_as_name']], train_df['label'])
test_df_encoded = target_encoder.transform(test_df[['attacker_as_name']])
val_df_encoded = target_encoder.transform(val_df[['attacker_as_name']])

train_df[['attacker_as_name']] = train_df_encoded.astype('float32')
test_df[['attacker_as_name']] = test_df_encoded.astype('float32')
val_df[['attacker_as_name']] = val_df_encoded.astype('float32')

del train_df_encoded, test_df_encoded, val_df_encoded, target_encoder
gc.collect()

0

### Attack type (OHE/Bin)

In [21]:
#Not sure if I want to use this yet

train_df.drop(['attack_type'], axis=1, inplace=True)
test_df.drop(['attack_type'], axis=1, inplace=True)
val_df.drop(['attack_type'], axis=1, inplace=True)

In [22]:
# Initialize a BinaryEncoder for the 'attack_service' column
binary_encoder = BinaryEncoder(cols=['attack_service'])

# Fit and transform on the training data
train_df_encoded = binary_encoder.fit_transform(train_df[['attack_service']])

# Transform the test data
test_df_encoded = binary_encoder.transform(test_df[['attack_service']])
val_df_encoded = binary_encoder.transform(val_df[['attack_service']])

# Merge the encoded columns back into the original DataFrames
train_df = pd.concat([train_df, train_df_encoded], axis=1)
test_df = pd.concat([test_df, test_df_encoded], axis=1)
val_df = pd.concat([val_df, val_df_encoded], axis=1)

# Clean up unnecessary variables
del train_df_encoded, test_df_encoded, val_df_encoded
gc.collect()

# Drop the original 'attack_service' column
train_df.drop('attack_service', axis=1, inplace=True)
test_df.drop('attack_service', axis=1, inplace=True)
val_df.drop('attack_service', axis=1, inplace=True)

# Cast to uint8 for efficiency
train_df['attack_service_0'] = train_df['attack_service_0'].astype('uint8')
test_df['attack_service_0'] = test_df['attack_service_0'].astype('uint8')
val_df['attack_service_0'] = val_df['attack_service_0'].astype('uint8')

train_df['attack_service_1'] = train_df['attack_service_1'].astype('uint8')
test_df['attack_service_1'] = test_df['attack_service_1'].astype('uint8')
val_df['attack_service_1'] = val_df['attack_service_1'].astype('uint8')

train_df['attack_service_2'] = train_df['attack_service_2'].astype('uint8')
test_df['attack_service_2'] = test_df['attack_service_2'].astype('uint8')
val_df['attack_service_2'] = val_df['attack_service_2'].astype('uint8')

train_df['attack_service_3'] = train_df['attack_service_3'].astype('uint8')
test_df['attack_service_3'] = test_df['attack_service_3'].astype('uint8')
val_df['attack_service_3'] = val_df['attack_service_3'].astype('uint8')

In [23]:
# Create a OneHotEncoder instance
encoder = OneHotEncoder(sparse=True, drop='first')  # Use sparse=True

# Fit and transform the 'attack_category' column in both train and test DataFrames
encoded_attack_category_train = encoder.fit_transform(train_df[['attack_category']]
                                                      )
# Convert the sparse matrices to dense arrays of integers
encoded_attack_category_train = encoded_attack_category_train.todense().astype('int8')

# Create DataFrames for the one-hot encoded 'attack_category' column
encoded_attack_category_train_df = pd.DataFrame(encoded_attack_category_train, columns=encoder.get_feature_names_out(['attack_category']))

# Concatenate the one-hot encoded 'attack_category' columns to the respective DataFrames
train_df = train_df.reset_index().drop(['index', 'attack_category'],axis=1).join(encoded_attack_category_train_df)

# Run GC to save memory
del encoded_attack_category_train, encoded_attack_category_train_df
gc.collect()

# Do it again for other 2 dataframes

encoded_attack_category_test = encoder.transform(test_df[['attack_category']])
encoded_attack_category_test = encoded_attack_category_test.todense().astype('int8')
encoded_attack_category_test_df = pd.DataFrame(encoded_attack_category_test, columns=encoder.get_feature_names_out(['attack_category']))
test_df = test_df.reset_index().drop(['index', 'attack_category'],axis=1).join(encoded_attack_category_test_df)
del encoded_attack_category_test, encoded_attack_category_test_df
gc.collect()

encoded_attack_category_val = encoder.transform(val_df[['attack_category']])
encoded_attack_category_val = encoded_attack_category_val.todense().astype('int8')
encoded_attack_category_val_df = pd.DataFrame(encoded_attack_category_val, columns=encoder.get_feature_names_out(['attack_category']))
val_df = val_df.reset_index().drop(['index', 'attack_category'],axis=1).join(encoded_attack_category_val_df)
del encoded_attack_category_val, encoded_attack_category_val_df
gc.collect()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0

### Attack time of day (OHE)

In [24]:
# Create a OneHotEncoder instance
encoder = OneHotEncoder(sparse=True, drop='first')  # Use sparse=True

# Fit and transform the 'attack_time_of_day' column in both train and test DataFrames
encoded_attack_time_of_day_train = encoder.fit_transform(train_df[['attack_time_of_day']]
                                                      )
# Convert the sparse matrices to dense arrays of integers
encoded_attack_time_of_day_train = encoded_attack_time_of_day_train.todense().astype('int8')

# Create DataFrames for the one-hot encoded 'attack_time_of_day' column
encoded_attack_time_of_day_train_df = pd.DataFrame(encoded_attack_time_of_day_train, columns=encoder.get_feature_names_out(['attack_time_of_day']))

# Concatenate the one-hot encoded 'attack_time_of_day' columns to the respective DataFrames
train_df = train_df.reset_index().drop(['index', 'attack_time_of_day'],axis=1).join(encoded_attack_time_of_day_train_df)

# Run GC to save memory
del encoded_attack_time_of_day_train, encoded_attack_time_of_day_train_df
gc.collect()

# Do it again for other 2 dataframes

encoded_attack_time_of_day_test = encoder.transform(test_df[['attack_time_of_day']])
encoded_attack_time_of_day_test = encoded_attack_time_of_day_test.todense().astype('int8')
encoded_attack_time_of_day_test_df = pd.DataFrame(encoded_attack_time_of_day_test, columns=encoder.get_feature_names_out(['attack_time_of_day']))
test_df = test_df.reset_index().drop(['index', 'attack_time_of_day'],axis=1).join(encoded_attack_time_of_day_test_df)
del encoded_attack_time_of_day_test, encoded_attack_time_of_day_test_df
gc.collect()

encoded_attack_time_of_day_val = encoder.transform(val_df[['attack_time_of_day']])
encoded_attack_time_of_day_val = encoded_attack_time_of_day_val.todense().astype('int8')
encoded_attack_time_of_day_val_df = pd.DataFrame(encoded_attack_time_of_day_val, columns=encoder.get_feature_names_out(['attack_time_of_day']))
val_df = val_df.reset_index().drop(['index', 'attack_time_of_day'],axis=1).join(encoded_attack_time_of_day_val_df)
del encoded_attack_time_of_day_val, encoded_attack_time_of_day_val_df
gc.collect()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0

### Checkpoint!

In [25]:
train_df.sample(5)

,watcher_as_name,attacker_as_name,watcher_uuid_enum,label,attack_day_sin,attack_day_cos,same_country,watcher_country_0,watcher_country_1,watcher_country_2,...,attack_service_1,attack_service_2,attack_service_3,attack_category_crawl,attack_category_exploit,attack_category_scan,attack_category_spam,attack_category_unknown,attack_time_of_day_day,attack_time_of_day_night
4637699,0.0222103465,0.0000579544,0.0128676472,0,-0.9749279022,-0.2225209326,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
981431,0.0160345342,0.0002759011,0.2599531710,0,0.9749279022,-0.2225209326,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
10081941,0.0150547987,0.0000000000,0.0251419302,0,0.7818315029,0.6234897971,0,0,1,0,...,0,0,1,0,0,0,1,0,0,1
32046144,0.0244245883,0.0023762495,0.0104358504,0,0.9749279022,-0.2225209326,0,0,1,1,...,0,0,1,0,0,0,1,0,0,1
14124055,0.0111621795,0.0000000000,0.0221366696,0,0.9749279022,-0.2225209326,0,0,1,0,...,0,0,1,0,1,0,0,0,0,1


# Model Stage

In [26]:
# I need all memory I can get
val_df.to_parquet('test_processed.parq')
del val_df
gc.collect()

0

### Hiperparam search

Metodología de la búsqueda: me guardé el score para los defaults, y luego puse rangos medio amplios alrededor de esos parámetros y le dije que me devolviera todos los parámetros que dieran mejor que eso.

Ahí a ojo pude observar ciertas tendencias de los parámetros con score "bueno" (`max_iter` de 4 cifras casi siempre, `alpha` alrededor de 1e-5, `tol` alrededor de 4e-3) y achicar los rangos para hacer una búsqueda más especializada.

Luego hice eso con todos los loss y me quedé con el mejor

In [ ]:
# Split the data into features (X) and target variable (y)
X_train = train_df.drop('label', axis=1)
y_train = train_df['label']

X_test = test_df.drop('label', axis=1)
y_test = test_df['label']

n_max_iter = 1000
batch_size = 6000000
cut = 0.5

for n_iter in range(n_max_iter):

  # Define hyperparameters
  params = {
    'alpha': random.uniform(1e-5, 1e-4),
    'penalty': 'elasticnet',
    'l1_ratio': random.uniform(0,1),
    'max_iter': random.randint(1000, 10000),
    'tol': random.uniform(1e-3, 1e-2),
    'loss': 'hinge',
    'shuffle': True
  }

  # Instantiate the SGDClassifier model
  sgd_model = SGDClassifier(
      alpha=params['alpha'],
      penalty=params['penalty'],
      l1_ratio=params['l1_ratio'],
      max_iter=params['max_iter'],
      tol=params['tol'],
      loss=params['loss'],
      shuffle=params['shuffle'],
      random_state=42
  )

  # Train the model in batches
  for i in range(0, len(X_train), batch_size):
      X_batch = X_train.iloc[i:i+batch_size]
      y_batch = y_train.iloc[i:i+batch_size]

      sgd_model.fit(X_batch, y_batch)

  # Make predictions on the test set
  y_pred = sgd_model.predict(X_test)

  # Evaluate the model
  f1_accuracy = f1_score(y_test, y_pred)

  if (f1_accuracy > cut):
    cut = f1_accuracy
    print(f'F1 Accuracy: {f1_accuracy}')
    print(f'Parameters: {params}')


### Search Results

RANDOM STATE 42

F1 Accuracy: 0.5399575518690173

Parameters: {'alpha': 9.728885639416088e-05, 'penalty': 'elasticnet', 'l1_ratio': 0.614730916366607, 'max_iter': 7988, 'tol': 0.002139720837939094, 'loss': 'hinge', 'shuffle': True}

---

F1 Accuracy: 0.5396219319940412

Parameters: {'alpha': 8.469281592683844e-05, 'penalty': 'elasticnet', 'l1_ratio': 0.08136166161048441, 'max_iter': 3469, 'tol': 0.0026458621715690508, 'loss': 'hinge'}

---

F1 Accuracy: 0.5389515651407688

Parameters: {'alpha': 5.417945695367806e-05, 'max_iter': 1930, 'tol': 0.0017631980426223016, 'loss': 'hinge'}

---

F1 Accuracy: 0.5388718466924104

Parameters: {'alpha': 8.939574131993856e-05, 'penalty': 'l1', 'max_iter': 9522, 'tol': 0.0077009771614071085, 'loss': 'hinge'}

---

F1 Accuracy: 0.5387163112891006

Parameters: {'alpha': 4.834698748925188e-05, 'max_iter': 5484, 'tol': 0.004784615699289584, 'loss': 'hinge'}

---

F1 Accuracy: 0.51813769135767

Parameters: {'alpha': 0.00032198363442605967, 'max_iter': 6547, 'tol': 0.006792767073748877, 'loss': 'epsilon_insensitive'}

---

F1 Accuracy: 0.5131948758912458

Parameters: {'alpha': 0.0005523982021547937, 'max_iter': 4689, 'tol': 0.0017825799430401961, 'loss': 'modified_huber'}

---

F1 Accuracy: 0.4881776893654056

Parameters: {'alpha': 2.389231494789442e-05, 'max_iter': 5855, 'tol': 0.0035301643113797845, 'loss': 'log_loss'}

---

F1 Accuracy: 0.4802747930798677

Parameters: {'alpha': 0.0001, 'max_iter': 1000, 'tol': 0.001, 'loss': 'log_loss'}

RANDOM STATE 317

F1 Accuracy: 0.628706532864497

Parameters: {'alpha': 9.728885639416088e-05, 'penalty': 'elasticnet', 'l1_ratio': 0.614730916366607, 'max_iter': 7988, 'tol': 0.002139720837939094, 'loss': 'hinge', 'shuffle': True}

---

F1 Accuracy: 0.6293167927108438

Parameters: {'alpha': 8.469281592683844e-05, 'penalty': 'elasticnet', 'l1_ratio': 0.08136166161048441, 'max_iter': 3469, 'tol': 0.0026458621715690508, 'loss': 'hinge'}

---

F1 Accuracy: 0.6170837560160785

Parameters: {'alpha': 5.417945695367806e-05, 'max_iter': 1930, 'tol': 0.0017631980426223016, 'loss': 'hinge'}

---

F1 Accuracy: 0.6396180132946353

Parameters: {'alpha': 8.939574131993856e-05, 'penalty': 'l1', 'max_iter': 9522, 'tol': 0.0077009771614071085, 'loss': 'hinge'}

---

F1 Accuracy: 0.6172968519973455

Parameters: {'alpha': 4.834698748925188e-05, 'max_iter': 5484, 'tol': 0.004784615699289584, 'loss': 'hinge'}

---

F1 Accuracy: 0.7142129532165727

Parameters: {'alpha': 0.00032198363442605967, 'max_iter': 6547, 'tol': 0.006792767073748877, 'loss': 'epsilon_insensitive'}

---

F1 Accuracy: 0.6062038762861602

Parameters: {'alpha': 0.0005523982021547937, 'max_iter': 4689, 'tol': 0.0017825799430401961, 'loss': 'modified_huber'}

---

F1 Accuracy: 0.6040129775561478

Parameters: {'alpha': 2.389231494789442e-05, 'max_iter': 5855, 'tol': 0.0035301643113797845, 'loss': 'log_loss'}

---

F1 Accuracy: 0.6019435825550077

Parameters: {'alpha': 0.0001, 'max_iter': 1000, 'tol': 0.001, 'loss': 'log_loss'}

### Kaggle submission

In [27]:
# Split the data into features (X) and target variable (y)
X_train = train_df.drop('label', axis=1)
y_train = train_df['label']

X_test = test_df.drop('label', axis=1)
y_test = test_df['label']

n_max_iter = 1000
batch_size = 6000000

# for n_iter in range(n_max_iter):

# Define hyperparameters
params = {
    'alpha': 9.728885639416088e-05,
    'penalty': 'elasticnet',
    'l1_ratio': 0.614730916366607,
    'max_iter': 7988,
    'tol': 0.002139720837939094,
    'loss': 'hinge',
    'shuffle': True
    }

# Instantiate the SGDClassifier model
sgd_model = SGDClassifier(
    alpha=params['alpha'],
    penalty=params['penalty'],
    l1_ratio=params['l1_ratio'],
    max_iter=params['max_iter'],
    tol=params['tol'],
    loss=params['loss'],
    shuffle=params['shuffle'],
    random_state=42
)

# Train the model in batches
for i in range(0, len(X_train), batch_size):
    X_batch = X_train.iloc[i:i+batch_size]
    y_batch = y_train.iloc[i:i+batch_size]

    sgd_model.fit(X_batch, y_batch)

# Make predictions on the test set
y_pred = sgd_model.predict(X_test)

# Evaluate the model
f1_accuracy = f1_score(y_test, y_pred)

print(f'F1 Accuracy: {f1_accuracy}')
print(f'Parameters: {params}')


F1 Accuracy: 0.7142129532165727
Parameters: {'alpha': 0.00032198363442605967, 'max_iter': 6547, 'tol': 0.006792767073748877, 'loss': 'epsilon_insensitive'}


In [28]:
val_df = pq.read_table('test_processed.parq').to_pandas()

In [ ]:
# in case I need them again
train_df = pq.read_table('train_encoded.parq').to_pandas()
test_df = pq.read_table('test_encoded.parq').to_pandas()

In [ ]:
train_df.to_parquet('train_encoded.parq')
test_df.to_parquet('test_encoded.parq')
del train_df, test_df
gc.collect()

In [29]:
kaggle_pred = sgd_model.predict(val_df)
kaggle_df = pd.DataFrame({'attacker_ip_enum': val_ips, 'prediction': kaggle_pred})

In [30]:
kaggle_df = kaggle_df.groupby('attacker_ip_enum').mean().reset_index()
kaggle_df['prediction'] = kaggle_df['prediction'].round().astype(int)

In [31]:
kaggle_df.set_index('attacker_ip_enum').to_csv('sgd_submission.csv')

# Conclusion stage

### ¿Cómo conviene elegir los datos de validación respecto de los de train?

> Conviene splitear por IPs para evitar target leaks. Gracias a que hice eso desde el inicio, ya con los encodings más simples tenía scores de 0.4 para arriba

### ¿Cuál es el mejor score en la competencia?

> Logré alcanzar un score de 0.51661 en la competencia tras probar varios de los hiperparámetros que encontré en la search. Los mejores parámetros para Kaggle NO son los mejores en el validation, probablemente debido a la distribución de 1's. El dataset de predicciones está [aquí]() [falta link]

> Curiosamente, el 0.7 de score en validación vino gracias a cambiar el `random_state` a la hora de hacer el split. No cambió mucho el score en kaggle (empeoró 2 puntos creo) igual. Pero la diferencia es muy grande, puede haber algún leak de por medio que esté causando estos resultados